# Получение данных

In [85]:
files = ["myciel3.col", "myciel7.col", "school1.col", "school1_nsh.col", "anna.col", "miles1000.col", "miles1500.col", "le450_5a.col", "le450_15b.col", "queen11_11.col"] # файлы, на которых должен быть протестирован код

In [86]:
data = {} 
# data - словарь вида 
# {"название датасета" : 
#     {"vertex_num": число вершин, 
#     "edge_num": число рёбер, 
#     "edges": 
#         {словарь вида вершина - список смежных ей вершин}
#     }
#  ...
# }

In [88]:
for file in files:
    data[file] = {"vertex_num": None, "edge_num": None, "edges": {}}
    with open("datasets/" + file, "r") as f: # открываем файл для чтения
        for row in f: # проходим по строкам
            if row[0] == "c": # если строка начинается с буквы "c" - это комментарий, пропускае строку
                continue
            elif row[0] == "p": # если строка начинается с буквы "p" - это описание проблемы, берём из этой строки число вершин и рёбер (последние два числа)
                data[file]["vertex_num"], data[file]["edge_num"] = int(row.split()[-2]), int(row.split()[-1])
            elif row[0] == "e": # если строка начинается с буквы "p" - это вершины, между которыми есть ребро
                v1, v2 = int(row.split()[-2]), int(row.split()[-1]) # запоминаем вершины

                # добавляем связь вершины v1 с v2
                if v1 not in data[file]["edges"].keys(): # если это первое упоминание вершины v1 - создадим для неё список с указанием v2
                    data[file]["edges"][v1] = [v2]
                elif v2 not in data[file]["edges"][v1]: # иначе - просто добавим v2 в список смежных вершин v1
                    data[file]["edges"][v1].append(v2)

                # аналогично, но относительно вершины v2
                if v2 not in data[file]["edges"].keys():
                    data[file]["edges"][v2] = [v1]
                elif v1 not in data[file]["edges"][v2]: # иначе - просто добавим v2 в список смежных вершин v1
                    data[file]["edges"][v2].append(v1)
        data[file]["edges"] = dict(sorted(data[file]["edges"].items())) # отсортируем вершины

In [91]:
data["myciel3.col"] # пример данных

{'vertex_num': 11,
 'edge_num': 20,
 'edges': {1: [2, 4, 7, 9],
  2: [1, 3, 6, 8],
  3: [2, 5, 7, 10],
  4: [1, 5, 6, 10],
  5: [3, 4, 8, 9],
  6: [2, 4, 11],
  7: [1, 3, 11],
  8: [2, 5, 11],
  9: [1, 5, 11],
  10: [3, 4, 11],
  11: [6, 7, 8, 9, 10]}}

# Реализация эвристик

### Вариант первый (Greedy sequential coloring) - просто раскрашиваем вершины одну за одной в минимально допустимое число цветов.

In [81]:
def sequential_coloring(edges: dict): # на вход - число вершин и список рёбер
    # памятка - число цветов не может превышать число вершин
    colors = [1] # цвета для окраски графа
    solution = {} # ответ

    uncolored_vertices = edges.keys() # список неокрашенных вершин

    for v1 in uncolored_vertices: # идём по неокрашенным вершинам
        possible_colors = colors.copy() # создаём список доступных для окраски v1 цветов

        for v2 in edges[v1]: # идём по смежным вершинам
            if solution.get(v2) is not None and solution[v2] in possible_colors: # проверяем, окрашены ли они
                possible_colors.remove(solution[v2]) # если окрашены, то удаляем твкой цвет из допустимых
        if len(possible_colors): # окрашиваем вершину в минимальный доступный цвет, если он имеется
            solution[v1] = possible_colors[0]
        else: # иначе - добавляем новый цвет и окрашиваем в него
            colors.append(colors[-1] + 1)
            solution[v1] = colors[-1]
        

    return len(colors), solution # возвращаем число цветов и соответствие вершин этим цветам

In [92]:
sequential_coloring(data["miles1500.col"]["edges"])

(76,
 {1: 1,
  2: 2,
  3: 2,
  4: 1,
  5: 3,
  6: 3,
  7: 4,
  8: 5,
  9: 4,
  10: 6,
  11: 5,
  12: 6,
  13: 7,
  14: 8,
  15: 7,
  16: 9,
  17: 6,
  18: 8,
  19: 10,
  20: 9,
  21: 10,
  22: 1,
  23: 11,
  24: 12,
  25: 13,
  26: 11,
  27: 14,
  28: 15,
  29: 12,
  30: 16,
  31: 13,
  32: 16,
  33: 17,
  34: 7,
  35: 18,
  36: 17,
  37: 15,
  38: 18,
  39: 19,
  40: 13,
  41: 20,
  42: 14,
  43: 21,
  44: 8,
  45: 22,
  46: 23,
  47: 20,
  48: 21,
  49: 19,
  50: 15,
  51: 11,
  52: 24,
  53: 25,
  54: 22,
  55: 26,
  56: 27,
  57: 28,
  58: 29,
  59: 20,
  60: 30,
  61: 26,
  62: 31,
  63: 23,
  64: 32,
  65: 27,
  66: 33,
  67: 34,
  68: 35,
  69: 36,
  70: 37,
  71: 38,
  72: 4,
  73: 39,
  74: 40,
  75: 41,
  76: 42,
  77: 24,
  78: 43,
  79: 44,
  80: 45,
  81: 25,
  82: 28,
  83: 46,
  84: 47,
  85: 26,
  86: 45,
  87: 29,
  88: 48,
  89: 49,
  90: 50,
  91: 51,
  92: 52,
  93: 53,
  94: 54,
  95: 10,
  96: 55,
  97: 30,
  98: 23,
  99: 56,
  100: 57,
  101: 58,
  102: 59,
  10

### Вариант второй (Smalest degree last with remove) - сначала отсортировываем все вершины в порядке уменьшения их степеней (с удалением), то есть - изначально рассматриваем вершину с наименьшей степенью (если таких несколько, то берём первую встреченную) и добавляем её в список, как бы полностью убирая её от других вершин (попутно уменьшая их степени). Идём таким образом по всем вершинам, добавляя их в список слева. В итоге получится список вида [вершина с наибольшей степенью, ..., вершина с наименьшей степенью]. Окраску начинаем с вершины наибольшей степени. (ожидается, что его применение даёт результат лучше, чем первого варианта).

In [ ]:
# def smalest_degree_last_with_remove:

# Результаты

### Ответы первого жадного алгоритма (Greedy sequential coloring):

<table>
  <tr>
    <th>Instance</th>
    <th>Time, sec</th>
    <th>Colors</th>
    <th>Color classes</th>
  </tr>
  <tr>
    <td>myciel3.col</td>
    <td></td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>myciel7.col</td>
    <td></td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>school1.col</td>
    <td></td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>school1_nsh.col</td>
    <td></td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>anna.col</td>
    <td></td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>miles1000.col</td>
    <td></td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>miles1500.col</td>
    <td></td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>le450_5a.col</td>
    <td></td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>le450_15b.col</td>
    <td></td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>queen11_11.col</td>
    <td></td>
    <td></td>
    <td></td>
  </tr>
</table>

### Ответы второго жадного алгоритма (Smalest degree last with remove):

<table>
  <tr>
    <th>Instance</th>
    <th>Time, sec</th>
    <th>Colors</th>
    <th>Color classes</th>
  </tr>
  <tr>
    <td>myciel3.col</td>
    <td></td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>myciel7.col</td>
    <td></td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>school1.col</td>
    <td></td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>school1_nsh.col</td>
    <td></td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>anna.col</td>
    <td></td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>miles1000.col</td>
    <td></td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>miles1500.col</td>
    <td></td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>le450_5a.col</td>
    <td></td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>le450_15b.col</td>
    <td></td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>queen11_11.col</td>
    <td></td>
    <td></td>
    <td></td>
  </tr>
</table>

Второй алгоритм показал более хорошие ответы по сравнению с первым ... (что-то на счёт времени)